# Flood Risk Areas

## Import Libraries


In [8]:
# import libraries
import pandas as pd
import geopandas as gpd
import requests
import warnings
warnings.filterwarnings('ignore')

## Prepare Data

### Import

In [5]:
# Load the shapefile
flood_risk_data_path = 'raw_data/Flood_Risk_Areas/Flood_Risk_AreasPolygon.shp'
flood_risk_data = gpd.read_file(flood_risk_data_path)

# Display the first few rows and the data types to understand the structure
flood_risk_data.head(), flood_risk_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   fra_id      189 non-null    object  
 1   fra_name    189 non-null    object  
 2   frr_cycle   189 non-null    int32   
 3   flood_sour  189 non-null    object  
 4   geometry    189 non-null    geometry
dtypes: geometry(1), int32(1), object(3)
memory usage: 6.8+ KB


(         fra_id                                    fra_name  frr_cycle  \
 0  UK04A0001ENG  Kingston upon Hull and Haltemprice, Humber          1   
 1  UK04A0002ENG      Leicester Principal Urban Area, Humber          1   
 2  UK04A0003ENG                       West Midlands, Humber          1   
 3  UK05A0001ENG                        South Essex, Anglian          1   
 4  UK06A0001ENG                              London, Thames          1   
 
       flood_sour                                           geometry  
 0  Surface Water  POLYGON ((510452.364 435813.679, 510539.836 43...  
 1  Surface Water  POLYGON ((462000 311000, 462000 310000, 463000...  
 2  Surface Water  POLYGON ((403000 295000, 404000 295000, 405000...  
 3  Surface Water  POLYGON ((566834.499 197462.696, 566857.404 19...  
 4  Surface Water  MULTIPOLYGON (((553000 180000, 553000 179000, ...  ,
 None)

In [7]:
flood_risk_data.head(20)

,fra_id,fra_name,frr_cycle,flood_sour,geometry
0,UK04A0001ENG,"Kingston upon Hull and Haltemprice, Humber",1,Surface Water,"POLYGON ((510452.364 435813.679, 510539.836 43..."
1,UK04A0002ENG,"Leicester Principal Urban Area, Humber",1,Surface Water,"POLYGON ((462000 311000, 462000 310000, 463000..."
2,UK04A0003ENG,"West Midlands, Humber",1,Surface Water,"POLYGON ((403000 295000, 404000 295000, 405000..."
3,UK05A0001ENG,"South Essex, Anglian",1,Surface Water,"POLYGON ((566834.499 197462.696, 566857.404 19..."
4,UK06A0001ENG,"London, Thames",1,Surface Water,"MULTIPOLYGON (((553000 180000, 553000 179000, ..."
5,UK04A0005ENG,"Beeston (Broxtowe), Humber",2,Rivers and Sea,"MULTIPOLYGON (((453690.437 337501.528, 453691...."
6,UK04A0006ENG,"Bentley, Humber",2,Rivers and Sea,"MULTIPOLYGON (((458299.996 406800.004, 458299...."
7,UK04A0007ENG,"Bingley, Humber",2,Rivers and Sea,"MULTIPOLYGON (((409965 441300.003, 409966.861 ..."
8,UK04A0008ENG,"Birmingham, Humber",2,Surface Water,"MULTIPOLYGON (((416799.996 292349.995, 416749...."
9,UK04A0009ENG,"City of Bradford, Humber",2,Surface Water,"POLYGON ((417849.997 438850, 417849.997 438800..."


In [11]:
flood_risk_data.shape

(189, 5)

In [54]:
# Define the API endpoint
api_url = "https://environment.data.gov.uk/flood-monitoring/id/floodAreas?&_limit=100000"

# Query the API
response = requests.get(url = api_url)

data = response.json()

print(data)

{'@context': 'http://environment.data.gov.uk/flood-monitoring/meta/context.jsonld', 'meta': {'publisher': 'Environment Agency', 'licence': 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/', 'documentation': 'http://environment.data.gov.uk/flood-monitoring/doc/reference', 'version': '0.9', 'comment': 'Status: Beta service', 'hasFormat': ['http://environment.data.gov.uk/flood-monitoring/id/floodAreas.csv?&_limit=100000', 'http://environment.data.gov.uk/flood-monitoring/id/floodAreas.rdf?&_limit=100000', 'http://environment.data.gov.uk/flood-monitoring/id/floodAreas.ttl?&_limit=100000', 'http://environment.data.gov.uk/flood-monitoring/id/floodAreas.html?&_limit=100000'], 'limit': 100000}, 'items': [{'@id': 'http://environment.data.gov.uk/flood-monitoring/id/floodAreas/053FWFPUWI06', 'county': 'Lincolnshire', 'description': 'Areas near the River Witham, Sincil Dyke, Great Gowts Drain, Fossdyke Canal, Boultham Drain and the Brayford Pool in Lincoln', 'eaAreaName': 

In [55]:
# Check if the request was successful
if response.status_code == 200:
    flood_data = response.json()
    # Extract relevant data
    flood_areas = flood_data.get('items', [])

    # Prepare the data
    flood_areas_list = []
    for area in flood_areas:
        flood_areas_list.append({
            '@id': area.get('@id'),
            'county': area.get('county'),
            'description': area.get('description'),
            'eaAreaName': area.get('eaAreaName'),
            'lat': area.get('lat'),
            'long': area.get('long'),
            'riverOrSea': area.get('riverOrSea'),
            'polygon': area.get('polygon')
        })

    # Convert to DataFrame
    flood_areas_df = pd.DataFrame(flood_areas_list)

    # Display the DataFrame
    print(flood_areas_df.head())
    print(flood_areas_df.shape)
else:
    print(f"Failed to retrieve data: {response.status_code}")

                                                 @id  \
0  http://environment.data.gov.uk/flood-monitorin...   
1  http://environment.data.gov.uk/flood-monitorin...   
2  http://environment.data.gov.uk/flood-monitorin...   
3  http://environment.data.gov.uk/flood-monitorin...   
4  http://environment.data.gov.uk/flood-monitorin...   

                                county  \
0                         Lincolnshire   
1                         Lincolnshire   
2                           Cumberland   
3  Cumberland, Westmorland and Furness   
4                           Lancashire   

                                         description              eaAreaName  \
0  Areas near the River Witham, Sincil Dyke, Grea...     Lincs and Northants   
1  Witham, Sincil Dyke, Great Gowts Drain, Fossdy...     Lincs and Northants   
2  Properties affected by surface water flooding ...  Cumbria and Lancashire   
3  River Derwent from Keswick to Bassenthwaite. T...  Cumbria and Lancashire   
4  Propert

In [56]:
flood_areas_df.head()

,@id,county,description,eaAreaName,lat,long,riverOrSea,polygon
0,http://environment.data.gov.uk/flood-monitorin...,Lincolnshire,"Areas near the River Witham, Sincil Dyke, Grea...",Lincs and Northants,53.22002,-0.55462,"River Witham, Sincil Dyke, Great Gowts Drain",http://environment.data.gov.uk/flood-monitorin...
1,http://environment.data.gov.uk/flood-monitorin...,Lincolnshire,"Witham, Sincil Dyke, Great Gowts Drain, Fossdy...",Lincs and Northants,53.21764,-0.55269,"River Witham, Sincil Dyke, Great Gowts Drain, Fos",http://environment.data.gov.uk/flood-monitorin...
2,http://environment.data.gov.uk/flood-monitorin...,Cumberland,Properties affected by surface water flooding ...,Cumbria and Lancashire,54.60153,-3.13183,River Greta,http://environment.data.gov.uk/flood-monitorin...
3,http://environment.data.gov.uk/flood-monitorin...,"Cumberland, Westmorland and Furness",River Derwent from Keswick to Bassenthwaite. T...,Cumbria and Lancashire,54.61737,-3.14762,"River Greta, St Johns Beck, Glendermackin",http://environment.data.gov.uk/flood-monitorin...
4,http://environment.data.gov.uk/flood-monitorin...,Lancashire,"Properties on Church Street, Keirby Walk, Bank...",Cumbria and Lancashire,53.79078,-2.24018,River Brun,http://environment.data.gov.uk/flood-monitorin...


In [ ]:
# Save the DataFrame to a CSV file
flood_areas_df.to_csv('flood_areas_data.csv', index=False)
print("Flood areas data saved to 'flood_areas_data.csv'")

In [44]:
# Define the API endpoint for flood warnings
api_url = "http://environment.data.gov.uk/flood-monitoring/id/floods"

# Query the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    flood_data = response.json()

    # Check if items are in the response
    if 'items' in flood_data:
        # Extract relevant data
        flood_warnings = flood_data['items']

        # Prepare the data
        flood_warnings_list = []
        for warning in flood_warnings:
            flood_warnings_list.append({
                'floodAreaID': warning.get('floodAreaID'),
                'floodAreaName': warning.get('floodAreaName'),
                'severity': warning.get('severity'),
                'severityLevel': warning.get('severityLevel'),
                'timeRaised': warning.get('timeRaised'),
                'timeMessageChanged': warning.get('timeMessageChanged'),
                'timeSeverityChanged': warning.get('timeSeverityChanged'),
                'message': warning.get('message'),
                'isTidal': warning.get('isTidal')
            })

        # Convert to DataFrame
        flood_warnings_df = pd.DataFrame(flood_warnings_list)

        # # Convert time columns to datetime
        # for col in ['timeRaised', 'timeMessageChanged', 'timeSeverityChanged']:
        #     flood_warnings_df[col] = pd.to_datetime(flood_warnings_df[col])

        # Filter the data for Merseyside and the date range from 2013 to 2023
        # flood_warnings_df = flood_warnings_df[
        #     (flood_warnings_df['timeRaised'] >= '2013-01-01') &
        #     (flood_warnings_df['timeRaised'] <= '2023-12-31') &
        #     (flood_warnings_df['floodAreaName'].str.contains(
        #         'Merseyside', case=False, na=False))
        # ]

        # Display the DataFrame
        print(flood_warnings_df.head())

        # Save the DataFrame to a CSV file
        flood_warnings_df.to_csv(
            'flood_warnings_merseyside_data.csv', index=False)
        print("Flood warnings data for Merseyside saved to 'flood_warnings_merseyside_data.csv'")
    else:
        print("No flood warning data found.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

Empty DataFrame
Columns: []
Index: []
Flood warnings data for Merseyside saved to 'flood_warnings_merseyside_data.csv'
